In [4]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


In [1]:
%cd yolov5/
%pip install -r requirements.txt

c:\Users\Henok\Desktop\Week7\KAIM-Week-7\notebooks\yolov5


C:\Users\Henok\AppData\Roaming\Python\Python313\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python.exe -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import torch
import cv2
import os

# Load pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Path to the directory containing the images
image_folder = '../data'

# Loop through the images in the folder and run object detection
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)

    # Check if the file is an image file (you might need to adjust the extensions)
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        print(f"Skipping {img_name} - not an image file.")
        continue  # Skip to the next file

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Check if image was loaded successfully
    if img is None:
        print(f"Error loading image: {img_path}")
        continue  # Skip to the next file

    # Run object detection
    results = model(img)

    # Show the results
    results.show()
    results.save(save_dir='detection_results/')

C:\Users\Henok\AppData\Roaming\Python\Python313\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Henok/.cache\torch\hub\master.zip


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Henok\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os
import pandas as pd
import torch

# Folder containing the images to run object detection on
image_folder = '../data'

# Step 1: Initialize the YOLO model (assuming you're using YOLOv5)
# The model is loaded from the Ultralytics YOLOv5 hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Step 2: Initialize an empty DataFrame to store all detection results
# We will append detection results for each image into this DataFrame
all_detections = pd.DataFrame()

# Step 3: Loop through each image in the folder and run object detection
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)

    # Check if the current item is a file and not a directory
    if not os.path.isfile(img_path):
        continue  # Skip directories or other non-file items

    # Step 4: Run object detection on the image
    results = model(img_path)

    # Step 5: Extract detection data (bounding box, class labels, and confidence scores)
    detections = results.pandas().xyxy[0]  # Pandas dataframe of detection results

    # Step 6: Add a new column for the image name so we can track which image the detections belong to
    detections['image_name'] = img_name

    # Print detections for debugging/verification
    print(f"Detections for {img_name}:")
    print(detections[['name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']])

    # Step 7: Append the detection results of the current image to the all_detections DataFrame
    all_detections = pd.concat([all_detections, detections], ignore_index=True)

# Step 8: Save all detection results into a single CSV file
# The file 'all_detections.csv' will contain the detections of all images processed
all_detections.to_csv('all_detections_new.csv', index=False)

print("All detection results saved to 'all_detections.csv'")

In [ ]:
import os
import logging
from pathlib import Path
import torch

# Step 1: Set up logging to track the process and capture errors
logging.basicConfig(filename='detection_process.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

logging.info('Starting object detection and result extraction...')

# Path to your folder containing the images
image_folder = '../data'

# Step 2: Initialize the YOLO model (assuming you're using YOLOv5)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Create a folder to store CSV detection results if it doesn't exist
output_csv_folder = 'detection_results'
Path(output_csv_folder).mkdir(exist_ok=True)

# Step 3: Process results to extract bounding box, class labels, and confidence scores
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)

    # Check if the current item is a file and not a directory
    if not os.path.isfile(img_path):
        logging.warning(f'Skipped non-file item: {img_name}')
        continue  # Skip directories or other non-file items

    try:
        # Run object detection on the image
        results = model(img_path)

        # Extract data from results (bounding box, confidence scores, class labels)
        detections = results.pandas().xyxy[0]  # Pandas dataframe of detection results

        logging.info(f"Detections for {img_name}: {len(detections)} objects found")

        # Display relevant columns
        print(f"Detections for {img_name}:")
        print(detections[['name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']])

        # Step 4: Save detections to CSV file
        csv_filename = os.path.join(output_csv_folder, f'{img_name}_detections.csv')
        detections[['name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']].to_csv(csv_filename, index=False)

        logging.info(f'Saved detections for {img_name} to {csv_filename}')

    except Exception as e:
        logging.error(f'Error processing {img_name}: {str(e)}')
        print(f"An error occurred with {img_name}: {str(e)}")

logging.info('Object detection and result storage process completed.')

In [ ]:
import os
import pandas as pd

# Folder containing all the individual detection CSV files
csv_folder = 'detection_results'

# List to store individual dataframes
dataframes = []

# Loop through each file in the folder
for csv_file in os.listdir(csv_folder):
    if csv_file.endswith('.csv'):  # Check if it's a CSV file
        csv_path = os.path.join(csv_folder, csv_file)

        # Read the CSV file into a dataframe
        df = pd.read_csv(csv_path)

        # Add a column to identify the image from which the detections came
        df['image_name'] = csv_file.replace('_detections.csv', '')

        # Append the dataframe to the list
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe to a new CSV file
merged_df.to_csv('merged_detections.csv', index=False)

print('All detections have been merged into merged_detections.csv')